Проект ведётся в соответствии со стандартом [CRISP-DM](https://en.wikipedia.org/wiki/Cross-industry_standard_process_for_data_mining)

# Анализ требований (Business Understanding)

**Бизнес-цель**: разработать модель, которая позволила бы агенству недвижимости обойти конкурентов по скорости и качеству совершения сделок.

**Data-Science цель**: разработать регрессионную модель прогнозирования стоимости жилья для агенства недвижимости, взяв за основу имеющийся [датасет](https://drive.google.com/file/d/11-ZNNIdcQ7TbT8Y0nsQ3Q0eiYQP__NIW/view?usp=share_link)
*(датасет также синхронизируется средствами DVC на google drive)*.

На этапе фактического внедрения модели необходимо прибегнуть к A/B-тестированию, сопоставив скорость и качество совершения сделок до внедрения модели и после интеграции модели прогнозирования стоимости жилья.
При этом уже сейчас можно было бы уточнить у заказчика, что он понимает под качеством совершения сделок, какие показатели у агенства сейчас и с какими конкурентами предполагается вести сравнение.

Ввиду отсутсвия на данный момент этой информации, можно предположить, что агенство недвижимости хочет максимизировать среднее отношение стоимости сделки $\text{Value}_\text{trans}$ ко времени совершения этой сделки $t_\text{trans}$ (*trans* от англ. *transaction*):

$$
    \mathbb{E}\left(\frac{\text{Value}_\text{trans}}{t_\text{trans}}\right) \rightarrow \text{max}
$$

Осуществить это можно, максимизируя точность предсказательной модели, так как при предсказании справедливой цены можно значительно ускорить завершение сделки (снизить $t_\text{trans}$), не потеряв, и, возможно, даже увеличив $\text{Value}_\text{trans}$, предотвращая демпинг цены на недвижимость.

Таким образом, это задача *регрессии*, для которой можно ввести следующие метрики качества модели, с учётом наличия истинных значений стоимости (обучение с учителем):
- Средняя абсолютная ошибка, выраженная в процентах (**MAPE**);
- Коэффициент детерминации (**$R^2$**).

Первая метрика позволит оценить, на сколько процентов в среднем предсказание отклоняется от реального значения.
Будем её использовать, так как нет сведений о том, какое значение целевого показателя считать приемлемым.

Вторая метрика покажет, насколько модель лучше, чем если бы все предсказания были бы средним по правильным ответам. 
То есть показывает, какую долю информации о зависимости (дисперсии) смогла уловить модель.

Однако, необходимо прежде оценить распределение целевого признака, может полезно прибегнуть к логарифмическим метрикам.

## Задачи

1. Провести разведывательный анализ (EDA) и очистку исходных данных.
Во многих признаках присутствуют дублирующиеся категории, ошибки ввода, жаргонные сокращения и так далее.
Необходимо отыскать закономерности, расшифровать все сокращения, найти синонимы в данных, обработать пропуски и удалить выбросы.

2. Выделить наиболее значимые факторы, влияющие на стоимость недвижимости (Data Preparation)

3. Построить модель для прогнозирования стоимости недвижимости.

4. Разработать веб-сервис, на вход которому поступают данные о некоторой выставленной на продажу недвижимости, а сервис прогнозирует его стоимость.


## Описание данных
- `status` — статус продажи;
- `private pool` и `PrivatePool` — наличие собственного бассейна;
- `propertyType` — тип объекта недвижимости;
- `street` — адрес объекта;
- `baths` — количество ванных комнат;
- `homeFacts` — сведения о строительстве объекта (содержит несколько
типов сведений, влияющих на оценку объекта);
- `fireplace` — наличие камина;
- `city` — город;
- `schools` — сведения о школах в районе;
- `sqft` — площадь в футах;
- `zipcode` — почтовый индекс;
- `beds` — количество спален;
- `state` — штат;
- `stories` — количество этажей;
- `mls-id` и `MlsId` — идентификатор MLS (Multiple Listing Service, система
мультилистинга);
- `target` — цена объекта недвижимости (целевой признак, который
необходимо спрогнозировать).

## Сведения о данных

1. Признаки `homeFacts` и `schools` представляют собой словари и
содержат несколько типов сведений. Необходимо десериализовать
содержимое этих признаков. Если в нём есть полезные данные,
необходимо создать из них новые признаки.

2. [Типы жилья и их основные характеристики](https://www.rocketmortgage.com/learn/types-of-houses)
(пригодится при обработке признака `propertyType`):
- apartment — съёмная квартира (нельзя купить);
- condo — кондоминиум (можно купить);
- co-op — квартира в жилищном кооперативе;
- single-family (detached, tiny home) — односемейный (отдельный,
маленький) дом;
- townhome, townhouse — таунхаус;
- cape cod — черепичная крыша, облицовка деревянным сайдингом,
дверь в центре здания, окна по бокам, один-два этажа;
- colonial home — минимум два этажа, симметрия, лестница в центре
здания, строгий внешний вид;
- contemporary — чистые, простые линии, нейтральные цвета, натуральные
текстуры;
- cottage — небольшая уютная веранда, небольшие жилые помещения;
- craftsman — деревянные элементы ручной работы, выступающие балки,
большие колонны;
- Greek revival — большие белые колонны, украшения в греческом стиле,
нарядный вход;
- farmhouse — прямоугольная планировка, большое крыльцо, амбарная
крыша;
- French country — влияние прованса, облицовка камнем, состаренный
вид;
- Mediterranean — белая штукатурка, дерево и камень в тёплых тонах,
черепичные крыши, элементы испанских и итальянских вилл;
- midcentury modern — сочетание плавных природных и строгих
геометрических линий, гладкость, лаконичность, большие окна;
- ranch — один этаж, низкая крыша, открытые жилые помещения, задний
двор;
- split-level — жилые помещения разделены короткими лестничными
пролётами;
- Tudor — деревянные балки, каменная кладка, асимметричная крутая
двускатная крыша;
- Victorian (Queen Anne Houses) — два-три и более этажей, богатый декор,
крутая двускатная крыша, небольшие башенки, яркий фасад;
- European Houses style — кирпич или камень, высокая крутая крыша,
высокие окна, часто со ставнями, традиционные декоративные детали
(фронтоны, замковые камни);
- log home — деревянный дом.

3. Некоторые жаргонные сокращения — для исправления дублирования
(пригодится при обработке признака `propertyType`):
- 'manufactured home (house)' = 'mobile' = 'prefab' = 'modular';
- 'mobile' = 'mo2 le' = 'mo2le';
- 'cabin' = 'ca2 n' = 'ca2n';
- midcentury = mid century = mid-century;
- single family = single-family home.

4. Некоторые термины, используемые в сфере купли-продажи
недвижимости:
- status — статус, состояние.
- estate — объект недвижимости.
- 'foreclosed' ('foreclosure') — «лишён права выкупа».
Это процесс, когда недвижимость переходит к кредитору из-за
неуплаты ипотеки заёмщиком. Продажа foreclosure-недвижимости
отличается от традиционных продаж.
- 'pre foreclosure' ('pre foreclosure auction') — «до выкупа»,
«аукцион до выкупа».
- 'under contract showing' ('under contract show', 'under
contract', 'active under contract') — предложение о покупке
уже сделано, но не принято продавцом окончательно, и
недвижимость продолжают показывать другим потенциальным
покупателям.
- 'under contract backups', 'active backup', 'backup contract' —
предложение о покупке уже сделано и принято продавцом,
однако он открыт для резервных предложений.
- 'contingency' — дополнительные обстоятельства (критерии,
которым должен соответствовать продавец/покупатель до
завершения сделки).
- 'contingency contract' — контракт на случай возникновения
дополнительных обстоятельств.
- 'active contingency' — сделка уже практически состоялась,
однако продавцу/покупателю для её завершения необходимо
выполнить какие-то условия.
- 'insp inspection contingency' — ситуация, когда покупатель
может официально проинспектировать объект недвижимости в
определённый период времени.
- 'pending escape clause' — договор, предполагающий свободный
выход сторон из него.
- 'pending backup wanted' — договор уже заключен, однако
продавец хочет продолжать показывать недвижимость другим
потенциальным покупателям и принимать резервные
предложения.
- 'pending take backups' — условие, предполагающее, что
продавец может принимать резерные предложения, если сделка
с текущим покупателем не состоится.
- 'pending continue show' — договор уже заключен, однако
продавец хочет продолжать показывать недвижимость другим
потенциальным покупателям на случай, если текущая сделка не
состоится.
- 'pending inspection' — «в ожидании инспекции».
- 'due diligence period' — период, в течение которого покупатель
имеет право проинспектировать недвижимость и изучить всю
важную для заключения сделки информацию.
- 'activated' — «активен», открыт для покупки.
- 'active with contract' — продавец уже принял предложение о
покупке, однако сделка ещё не закрыта.
- 'active with offer' — продавец принял предложение о покупке,
однако ищет резервные предложения на случай, если сделка с
текущим покупателем не состоится.
- 'active auction', 'auction active', 'auction' — продажа
недвижимости с помощью аукциона.
- cooling-off period — период, в течение которого покупатель
может расторгнуть договор после завершения сделки.


# Импорт библиотек и данных

In [1]:
import sys
import os
import pandas as pd
from pathlib import Path
import pickle

from ydata_profiling import ProfileReport

from IPython.display import display, HTML

root_folder = '../'
sys.path.append(root_folder)
from src.data import make_dataset

source_data = Path(root_folder, 'data', 'raw', 'data.csv')

In [2]:
df = pd.read_csv(
    source_data
)
df.info()
display(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103114 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226469 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377180,NaN,NaN,Single Family,20800 NE 23rd Ave,6.0,"{'atAGlanceFacts': [{'factValue': '1990', 'fac...",NaN,Miami,"[{'rating': ['10/10', '5/10'], 'data': {'Dista...","4,017",33180,5,FL,0.0,NaN,Yes,A10702700,"$1,249,000"
377181,for sale,NaN,condo,3530 N Lake Shore Dr #4B,3 Baths,"{'atAGlanceFacts': [{'factValue': '1924', 'fac...",NaN,Chicago,"[{'rating': ['1/10', '5/10', '7/10'], 'data': ...","2,000 sqft",60657,3 Beds,IL,9.0,NaN,NaN,10374233,"$674,999"
377182,for sale,NaN,single-family home,15509 Linden Blvd,3 Baths,"{'atAGlanceFacts': [{'factValue': '1950', 'fac...",NaN,Jamaica,"[{'rating': ['5/10', '4/10'], 'data': {'Distan...","1,152 sqft",11434,3 Beds,NY,2,NaN,NaN,NaN,"$528,000"
377183,NaN,NaN,NaN,7810 Pereida St,NaN,"{'atAGlanceFacts': [{'factValue': None, 'factL...",NaN,Houston,"[{'rating': ['NA', 'NA', 'NA'], 'data': {'Dist...",NaN,77028,"8,479 sqft",TX,NaN,NaN,NaN,NaN,"$34,500"


# Первичный анализ данных (Data Understanding)

Проведём первичный анализ данных, используя `ydata_profiling` и выведенную информацию выше

In [3]:
profile_path = Path(root_folder, 'reports', 'init_profiling.html')

# Create init profiling report if it does not exist
if not os.path.isfile(profile_path):
    profile = ProfileReport(df, title='House profiling')
    profile.to_file(profile_path)
    print('Save to:', profile_path)
print('See report in ', profile_path)

See report in  ../reports/init_profiling.html


Некоторые наблюдения:

- Имеется датасет из 377185 строк и 18 колонок
- Все колонки имеют тип "object" и требуют предобработки для того, чтобы получить численные значения, где они предполагаются
- Имеем дело с большим количеством пропусков в ряде колонок
- Имеются несколько дублирующихся строчек (порядка 49)
- Некоторые признаки дублируются
- Пропуски в `target` критичны для нас, поэтому имеет смысл удалить строки с пропусками в целевом признаке

Будем разбираться по-порядку, анализируя каждый признак по отдельности

## Target

Приведём таргет к числовому типу. 
Для этого посмотрим на нечисловые символы

In [26]:
df['target'].str.findall(r'[^0-9]').value_counts()

target
[$, ,]                            267483
[,]                                55868
[$, ,, ,]                          33614
[,, ,]                             10001
[$, ,, +]                           6931
[$, ,, /, m, o]                      373
[$, ,, ,, +]                         332
[$]                                   64
[$, /, m, o]                          24
[]                                    13
[$, ,,  , -,  , $, ,, /, m, o]         1
Name: count, dtype: int64

In [30]:
df[df['target'].str.contains('m', na=False)]['target']

547        $5,500/mo
609       $10,500/mo
2075       $6,390/mo
3025       $1,200/mo
3645       $3,600/mo
             ...    
371791       $890/mo
372459     $2,500/mo
374288     $2,000/mo
375550     $1,500/mo
376976     $1,900/mo
Name: target, Length: 398, dtype: object

In [14]:
def get_numerical_target(target:pd.Series):
    target = target.dropna()
    target = target.apply(lambda x: x[1])
    # target = target.str.replace(',', '')
    return target

get_numerical_target(df['target'])

IndexError: string index out of range

In [40]:
res = df['homeFacts'][1].replace('\'', '"').replace('None', '""')
res[285:290]

'l": "'

In [41]:
res

'{"atAGlanceFacts": [{"factValue": "2019", "factLabel": "Year built"}, {"factValue": "", "factLabel": "Remodeled year"}, {"factValue": "", "factLabel": "Heating"}, {"factValue": "", "factLabel": "Cooling"}, {"factValue": "", "factLabel": "Parking"}, {"factValue": "5828 sqft", "factLabel": "lotsize"}, {"factValue": "$159/sqft", "factLabel": "Price/sqft"}]}'

In [44]:
import json
facts_dt = json.loads(res)
facts_dt

{'atAGlanceFacts': [{'factValue': '2019', 'factLabel': 'Year built'},
  {'factValue': '', 'factLabel': 'Remodeled year'},
  {'factValue': '', 'factLabel': 'Heating'},
  {'factValue': '', 'factLabel': 'Cooling'},
  {'factValue': '', 'factLabel': 'Parking'},
  {'factValue': '5828 sqft', 'factLabel': 'lotsize'},
  {'factValue': '$159/sqft', 'factLabel': 'Price/sqft'}]}

In [45]:
facts_dt['atAGlanceFacts']

[{'factValue': '2019', 'factLabel': 'Year built'},
 {'factValue': '', 'factLabel': 'Remodeled year'},
 {'factValue': '', 'factLabel': 'Heating'},
 {'factValue': '', 'factLabel': 'Cooling'},
 {'factValue': '', 'factLabel': 'Parking'},
 {'factValue': '5828 sqft', 'factLabel': 'lotsize'},
 {'factValue': '$159/sqft', 'factLabel': 'Price/sqft'}]

In [47]:
df['mls-id'].value_counts()

mls-id
No MLS#      3
No           3
1498550      2
39888954     2
608063       2
            ..
1020314      1
A10762436    1
1592770      1
14201834     1
F10202858    1
Name: count, Length: 24907, dtype: int64

In [48]:
df['MlsId'].value_counts()

MlsId
NO MLS                     24
No MLS #                   16
 A, Houston, TX 77008      13
 12A, Orlando, FL 32833    11
 B, Houston, TX 77008       9
                           ..
19092240                    1
RX-10563061                 1
218080001                   1
14154778                    1
10374233                    1
Name: count, Length: 232944, dtype: int64

In [121]:
df[df['MlsId'].str.lower().str.contains(r'no[^a-zA-Z0-9,](mls)*', na=False)]['MlsId'].value_counts()

/var/folders/dv/7ymr80q90cs45p34fgkdmj0c0000gn/T/ipykernel_91802/3364708247.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[df['MlsId'].str.lower().str.contains(r'no[^a-zA-Z0-9,](mls)*', na=False)]['MlsId'].value_counts()


MlsId
NO MLS                                      24
No MLS #                                    16
No                                           9
No mls                                       8
No MLS                                       6
No MLS                                       2
0087, Solano at Altair, Irvine, CA 92618     1
0061, Solano at Altair, Irvine, CA 92618     1
No  MLs #                                    1
Name: count, dtype: int64

In [88]:
df[df['MlsId'].str.lower().str.contains(r'no(\W*)(mls)|(no)^[A-Za-z]', na=False)]['MlsId'].value_counts()

/var/folders/dv/7ymr80q90cs45p34fgkdmj0c0000gn/T/ipykernel_91802/2669860384.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[df['MlsId'].str.lower().str.contains(r'no(\W*)(mls)|(no)^[A-Za-z]', na=False)]['MlsId'].value_counts()


MlsId
NO MLS       24
No MLS #     16
No mls        8
No MLS        6
NOMLSID       6
No MLS        2
No  MLs #     1
Name: count, dtype: int64

In [96]:
df[df['MlsId'].str.lower().str.contains('no', na=False)]['MlsId']

2348               1401, North Miami, FL 33181
6720                                       No 
8196                                    No MLS
8610                                       No 
9028                                    No MLS
                          ...                 
364310                      16, Reno, NV 89523
366774     17208W, North Miami Beach, FL 33162
367932                                     No 
370434             2207, North Miami, FL 33181
372018        301, North Bay Village, FL 33141
Name: MlsId, Length: 161, dtype: object

In [102]:
df[df['MlsId'].isin(['no'])]['MlsId']

Series([], Name: MlsId, dtype: object)

In [116]:
df[df['MlsId'].str.lower().str.strip().isin(['no'])]['MlsId']

6720      No 
8610      No 
44792     No 
132144    No 
207894    No 
223985    No 
326882    No 
344235    No 
367932    No 
Name: MlsId, dtype: object